<a href="https://colab.research.google.com/github/leonfrench/BIODEP_CRP/blob/main/BIODEP_nonhuman_reads_R.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
library(dplyr)
library(magrittr)
library(readr)
library(tidyr)
library(progress)
library(broom)
library(ggplot2)


Attaching package: ‘tidyr’


The following object is masked from ‘package:magrittr’:

    extract




In [3]:
#from google bigquery STAT tables
# SELECT m.acc, m.sample_acc, m.biosample, m.sra_study, m.bioproject, tax_id, name, total_count, self_count, mbases
# FROM `nih-sra-datastore.sra.metadata` as m, `nih-sra-datastore.sra_tax_analysis_tool.tax_analysis` as tax
# WHERE m.acc=tax.acc AND m.sra_study = 'ERP163445'

stat_table <- read_csv("https://github.com/leonfrench/BIODEP_CRP/raw/refs/heads/main/data/bq-results-20250329-184723-1743274056865.csv")
stat_table %<>% select(acc, biosample, tax_id, name, total_count)



Rows: 464272 Columns: 10
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (6): acc, sample_acc, biosample, sra_study, bioproject, name
dbl (4): tax_id, total_count, self_count, mbases

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
#number of samples
stat_table %>% pull(biosample) %>% unique() %>% length()

[1] 560

In [24]:
#samples with Porphyromonas gingivalis (transient resident of the blood microbiome)
stat_table %>% filter(grepl("Porphyromonas gingivalis", name)) %>% nrow()

[1] 14

In [6]:
sra_table <- read_csv("https://github.com/leonfrench/BIODEP_CRP/raw/refs/heads/main/data/SraRunTable.csv")

Rows: 560 Columns: 43
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr  (33): Run, Assay Type, batch, BioProject, BioSample, Center Name, commo...
dbl   (5): AvgSpotLen, Bases, Bytes, collection_date, version
dttm  (2): ReleaseDate, create_date
date  (3): ENA-FIRST-PUBLIC (run), ENA_first_public, ENA-LAST-UPDATE (run)

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [7]:
sra_table %>% pull(BioSample) %>% unique() %>% length()


[1] 560

In [8]:
target_cols <- c('Bases','batch','collection_date','diagnosis','Experiment','sample_name','treatment_outcome','crp')

In [9]:
sra_table %<>% select(acc = Run, all_of(target_cols))
full_table <- inner_join(sra_table, stat_table)

full_table %<>% mutate(crp = as.numeric(gsub("\\\\,", ".", crp)))
full_table %<>% mutate(biosample = gsub("_L[^_]*$", "", sample_name))


Joining with `by = join_by(acc)`


In [10]:
full_table %<>% filter(!is.na(crp))

In [11]:
#should be 168 individuals
full_table %>% pull(sample_name) %>% unique() %>% length()
full_table %>% pull(biosample) %>% unique() %>% length()

[1] 556

[1] 139

In [12]:
#105 with MDD and 34 controls
full_table %>% select(biosample, diagnosis) %>% distinct() %>% group_by(diagnosis) %>% count()

diagnosis,n
<chr>,<int>
HC,34
MDD,105


In [31]:
#samples with Porphyromonas gingivalis (transient resident of the blood microbiome)
full_table %>% filter(name == "Porphyromonas gingivalis") %>% pull(biosample) %>% unique() %>% length()

[1] 11

In [13]:
sample_counts <- full_table %>% select(biosample, all_of(target_cols)) %>% distinct() %>%
  group_by(across(all_of(c("biosample", "batch", "diagnosis", "crp" )))) %>%
  summarize(n = n(), Bases = sum(Bases))
sample_counts %<>% rename(run_count = n)

`summarise()` has grouped output by 'biosample', 'batch', 'diagnosis'. You can
override using the `.groups` argument.


In [14]:
sample_counts %>% head()

biosample,batch,diagnosis,crp,run_count,Bases
<chr>,<chr>,<chr>,<dbl>,<int>,<dbl>
101_S2,Batch8,MDD,0.2,4,3073469866
102_S3,Batch8,MDD,9.4,4,3041203169
103_S4,Batch8,MDD,3.9,4,2946558970
104_S5,Batch8,HC,1.7,4,2709224619
105_S6,Batch8,HC,0.4,4,3354212144
106_S7,Batch8,MDD,4.0,4,2645873905


In [15]:
#test for an association between number of organisms detected and the factors
for_lm <- inner_join(sample_counts, full_table %>% group_by(across(all_of(c("biosample", "batch", "diagnosis", "crp")))) %>% count())
for_lm %<>% mutate(detect_per_base = n/Bases*1000000000) %>% arrange(detect_per_base)

Joining with `by = join_by(biosample, batch, diagnosis, crp)`


In [16]:
summary(lm(data = for_lm, detect_per_base ~ Bases + batch + diagnosis + crp))


Call:
lm(formula = detect_per_base ~ Bases + batch + diagnosis + crp, 
    data = for_lm)

Residuals:
    Min      1Q  Median      3Q     Max 
-1043.5  -298.2   -95.4   131.2  3444.5 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
(Intercept)   2.090e+03  2.868e+02   7.286 3.08e-11 ***
Bases        -4.252e-07  7.034e-08  -6.045 1.58e-08 ***
batchBatch10  5.553e+02  2.270e+02   2.446   0.0158 *  
batchBatch2   1.478e+02  2.288e+02   0.646   0.5196    
batchBatch3   1.189e+03  2.248e+02   5.290 5.24e-07 ***
batchBatch4   4.460e+02  2.251e+02   1.982   0.0497 *  
batchBatch5   3.784e+02  2.257e+02   1.677   0.0960 .  
batchBatch6   1.273e+02  2.242e+02   0.568   0.5712    
batchBatch7   2.753e+02  2.244e+02   1.227   0.2221    
batchBatch8   4.713e+01  2.305e+02   0.204   0.8383    
batchBatch9   5.215e+02  2.265e+02   2.302   0.0229 *  
diagnosisMDD  4.488e+01  1.230e+02   0.365   0.7159    
crp          -1.308e+01  2.321e+01  -0.563   0.5742    
---
Signif. code

In [17]:
summary(lm(data = for_lm, n ~ Bases + batch + diagnosis + crp))


Call:
lm(formula = n ~ Bases + batch + diagnosis + crp, data = for_lm)

Residuals:
    Min      1Q  Median      3Q     Max 
-2165.0  -668.1  -271.7   331.1  7090.1 

Coefficients:
               Estimate Std. Error t value Pr(>|t|)    
(Intercept)   1.321e+03  7.082e+02   1.865  0.06450 .  
Bases         2.723e-07  1.737e-07   1.568  0.11950    
batchBatch10  1.244e+03  5.605e+02   2.219  0.02828 *  
batchBatch2   2.578e+02  5.651e+02   0.456  0.64900    
batchBatch3   2.656e+03  5.551e+02   4.785  4.7e-06 ***
batchBatch4   1.602e+03  5.558e+02   2.882  0.00464 ** 
batchBatch5   1.337e+03  5.573e+02   2.399  0.01790 *  
batchBatch6   5.868e+02  5.537e+02   1.060  0.29129    
batchBatch7   1.194e+03  5.541e+02   2.156  0.03302 *  
batchBatch8   6.219e+02  5.693e+02   1.092  0.27676    
batchBatch9   1.609e+03  5.593e+02   2.878  0.00471 ** 
diagnosisMDD -5.403e+00  3.038e+02  -0.018  0.98584    
crp           9.924e+00  5.731e+01   0.173  0.86281    
---
Signif. codes:  0 ‘***’ 0.001 ‘

In [32]:
# -------------------------------------------------------------------------------
# Create `combined_results` summarizing per‑species abundance and modeling stats.
# For each unique microbial species (“name”) in `full_table`:
#   1. Aggregate read counts per biosample.
#   2. Join with `sample_counts` to add total sequenced bases and other metadata.
#   3. Replace missing counts with zero and compute a bases‑normalized metric.
#   4. Fit a linear model (total_count ~ Bases + batch + diagnosis + crp).
#   5. Record adjusted R², term‑specific p‑values, detection frequency, and the
#      sum of counts for the species.
# Output: `combined_results`, one row per species with model statistics.
# -------------------------------------------------------------------------------

combined_results <- tibble()

for (target_species in unique(full_table$name)) {

  species_data <- full_table %>%
    filter(name == target_species) %>%
    select(biosample, total_count) %>%
    group_by(biosample) %>%
    summarise(total_count = sum(total_count), .groups = "drop") %>%
    right_join(sample_counts, by = join_by(biosample)) %>%
    mutate(
      total_count     = replace_na(total_count, 0),
      base_normalized = total_count / Bases * 1e9
    )

  model <- summary(lm(total_count ~ Bases + batch + diagnosis + crp, species_data))

  single_row <- tibble(
    target_species = target_species,
    modelr2        = model$adj.r.squared,
    detections     = species_data %>% filter(total_count != 0) %>% nrow(),
    sum_counts     = sum(species_data$total_count)
  ) %>%
    bind_cols(
      broom::tidy(model) %>%
        select(term, p.value) %>%
        pivot_wider(names_from = term, values_from = p.value)
    )

  combined_results <- bind_rows(combined_results, single_row)

  if (nrow(combined_results) %% 500 == 0) {
    message(sprintf("Processed %d / %d organisms",
    nrow(combined_results), unique(full_table %>% pull(name)) %>% length()))
  }
}

Processed 500 / 16198 organisms

Processed 1000 / 16198 organisms

Processed 1500 / 16198 organisms

Processed 2000 / 16198 organisms

Processed 2500 / 16198 organisms

Processed 3000 / 16198 organisms

Processed 3500 / 16198 organisms

Processed 4000 / 16198 organisms

Processed 4500 / 16198 organisms

Processed 5000 / 16198 organisms

Processed 5500 / 16198 organisms

Processed 6000 / 16198 organisms

Processed 6500 / 16198 organisms

Processed 7000 / 16198 organisms

Processed 7500 / 16198 organisms

Processed 8000 / 16198 organisms

Processed 8500 / 16198 organisms

Processed 9000 / 16198 organisms

Processed 9500 / 16198 organisms

Processed 10000 / 16198 organisms

Processed 10500 / 16198 organisms

Processed 11000 / 16198 organisms

Processed 11500 / 16198 organisms

Processed 12000 / 16198 organisms

Processed 12500 / 16198 organisms

Processed 13000 / 16198 organisms

Processed 13500 / 16198 organisms

Processed 14000 / 16198 organisms

Processed 14500 / 16198 organisms

Proce

In [33]:
combined_results %>% arrange(`crp`) %>% head(20) %>% select(target_species, detections, sum_counts, crp, diagnosisMDD)

target_species,detections,sum_counts,crp,diagnosisMDD
<chr>,<int>,<dbl>,<dbl>,<dbl>
Thiopseudomonas denitrificans,6,14,6.074448e-07,0.9218365
Myoviridae sp. ctVKV3,2,2,1.349563e-05,0.8858679
Mycolicibacterium bacteremicum,2,3,1.676431e-05,0.8768499
Aspergillus subgen. Aspergillus,2,3,1.676431e-05,0.8768499
Aspergillus chevalieri,2,3,1.676431e-05,0.8768499
Haemophilus sputorum,30,590,2.702328e-05,0.9880077
Fusobacterium polymorphum ATCC 10953,4,6,2.804419e-05,0.5273935
Microbacterium hibisci,4,4,4.662722e-05,0.9173514
Streptococcus oralis,88,584,5.020735e-05,0.9754413


In [34]:
combined_results %>% arrange(`diagnosisMDD`) %>% head(20) %>% select(target_species, detections, sum_counts, crp, diagnosisMDD)

target_species,detections,sum_counts,crp,diagnosisMDD
<chr>,<int>,<dbl>,<dbl>,<dbl>
Nocardioides marmoribigeumensis,33,169,0.78563156,0.0004760352
Acinetobacter bouvetii,82,492,0.50732831,0.0018301993
Oceanobacillus iheyensis,3,3,0.89294361,0.0020533592
Nairoviridae,110,449,0.65887524,0.0024076389
Orthonairovirus,110,449,0.65887524,0.0024076389
Orthonairovirus haemorrhagiae,110,449,0.65887524,0.0024076389
Halpernia,13,31,0.02460082,0.0024342913
Legionella jamestowniensis,3,3,0.86297935,0.0025720152
Melghirimyces,4,6,0.43719710,0.0027217383


In [36]:
combined_results %>% head()
colnames(combined_results)


target_species,modelr2,detections,sum_counts,(Intercept),Bases,batchBatch10,batchBatch2,batchBatch3,batchBatch4,batchBatch5,batchBatch6,batchBatch7,batchBatch8,batchBatch9,diagnosisMDD,crp
<chr>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
cellular organisms,0.9983954,139,2773867126,6.413165e-01,8.269416e-176,2.171541e-01,0.1594096,2.818031e-02,7.441269e-03,4.913262e-01,9.360912e-02,9.301878e-01,5.811970e-01,9.261220e-01,0.13688588,0.85833313
Amniota,0.9887740,139,2554202563,6.528207e-01,7.812851e-123,8.511117e-01,0.8066524,8.966680e-03,4.197384e-01,6.842731e-01,1.373458e-01,7.736521e-01,5.067890e-01,5.751578e-02,0.95892705,0.71753879
Caniformia,0.3387111,139,81403,2.653095e-01,4.454264e-09,5.892502e-01,0.6166287,6.033077e-02,1.158998e-02,1.675593e-01,3.953240e-01,6.105378e-01,8.145545e-01,6.319679e-01,0.68148969,0.81948676
Cercopithecidae,0.6323060,139,2915938,7.458141e-01,7.611303e-27,1.975866e-01,0.1195979,3.901192e-01,2.564989e-01,4.701160e-01,9.143186e-01,4.524561e-01,4.662652e-01,5.068155e-01,0.28612572,0.51826955
Muridae,0.7468732,46,8516426,1.171321e-12,1.690774e-01,8.527691e-24,0.1637200,4.509626e-17,3.140207e-22,6.824605e-22,2.561578e-23,9.606248e-23,1.434091e-21,6.520249e-23,0.01636825,0.07692084
Mammalia,0.9891397,139,2521345029,6.960989e-01,9.819234e-124,9.778476e-01,0.6597712,4.201310e-03,3.884398e-01,5.206467e-01,1.171887e-01,8.736807e-01,5.816381e-01,4.657483e-02,0.86421073,0.68298564


[1] "target_species" "modelr2"        "detections"     "sum_counts"    
 [5] "(Intercept)"    "Bases"          "batchBatch10"   "batchBatch2"   
 [9] "batchBatch3"    "batchBatch4"    "batchBatch5"    "batchBatch6"   
[13] "batchBatch7"    "batchBatch8"    "batchBatch9"    "diagnosisMDD"  
[17] "crp"

In [39]:
#look at two species
combined_results %>% filter(target_species %in% c("Homo sapiens", "Porphyromonas gingivalis"))

target_species,modelr2,detections,sum_counts,(Intercept),Bases,batchBatch10,batchBatch2,batchBatch3,batchBatch4,batchBatch5,batchBatch6,batchBatch7,batchBatch8,batchBatch9,diagnosisMDD,crp
<chr>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Homo sapiens,0.607103226,139,192011249,0.8674143,5.821479e-24,0.6359393,0.4552672,0.1388725,0.007714065,0.38232655,0.3858178,0.6606225,0.6547915,0.9778466,0.2166534,0.7276608
Porphyromonas gingivalis,-0.003541731,11,22,0.9552877,7.303972e-01,0.9376490,0.9199658,0.9415337,0.191193802,0.05763167,0.8865799,0.4771167,0.2995888,0.4426183,0.2619778,0.8684023


In [22]:
full_table %>% filter(name == "Streptococcus oralis") %>%
  select(biosample, total_count) %>%
  group_by(biosample) %>%
  summarise(total_count = sum(total_count), .groups = "drop") %>%
  right_join(sample_counts, by = join_by(biosample)) %>%
  mutate(
    total_count     = replace_na(total_count, 0),
    base_normalized = total_count / Bases * 1e9
  ) %>% arrange(-total_count) %>% head(10)


biosample,total_count,batch,diagnosis,crp,run_count,Bases,base_normalized
<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<dbl>
61_S5,135,Batch5,MDD,10.3,4,3039583236,44.413984
53_S10,73,Batch4,MDD,8.4,4,3233602084,22.575443
92_S4,21,Batch7,MDD,0.3,4,2870762943,7.315129
2749_S7,19,Batch10,MDD,0.8,4,5565898825,3.413645
60_S4,19,Batch5,MDD,0.6,4,2650842653,7.167532
9_S9,18,Batch1,HC,2.2,4,3582839894,5.023948
105_S6,14,Batch8,HC,0.4,4,3354212144,4.173856
13_S12,14,Batch1,MDD,0.3,4,4165627086,3.360839
94_S6,14,Batch7,MDD,3.6,4,2774008462,5.046848


In [23]:
full_table %>% filter(name == "Haemophilus sputorum") %>%
  select(biosample, total_count) %>%
  group_by(biosample) %>%
  summarise(total_count = sum(total_count), .groups = "drop") %>%
  right_join(sample_counts, by = join_by(biosample)) %>%
  mutate(
    total_count     = replace_na(total_count, 0),
    base_normalized = total_count / Bases * 1e9
  ) %>% arrange(-total_count) %>% head(10)

biosample,total_count,batch,diagnosis,crp,run_count,Bases,base_normalized
<chr>,<dbl>,<chr>,<chr>,<dbl>,<int>,<dbl>,<dbl>
61_S5,332,Batch5,MDD,10.3,4,3039583236,109.2255004
53_S10,161,Batch4,MDD,8.4,4,3233602084,49.7896760
60_S4,27,Batch5,MDD,0.6,4,2650842653,10.1854405
36_S1,22,Batch4,MDD,1.3,4,2050120332,10.7310774
54_S11,15,Batch4,MDD,3.3,4,3183554164,4.7117150
8654_S2,4,Batch10,MDD,0.8,4,2779599645,1.4390562
106_S7,2,Batch8,MDD,4.0,4,2645873905,0.7558939
128_S13,2,Batch7,MDD,0.8,4,3059309372,0.6537423
47_S5,2,Batch4,MDD,3.1,4,2607688920,0.7669626
